<a href="https://colab.research.google.com/github/hoanghuumanh/AI_project/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip "/content/drive/My Drive/train movielen10k/ml-100k.zip" -d "/content/drive/My Drive/train movielen10k"



Archive:  /content/drive/My Drive/train movielen10k/ml-100k.zip
   creating: /content/drive/My Drive/train movielen10k/ml-100k/
  inflating: /content/drive/My Drive/train movielen10k/ml-100k/allbut.pl  
  inflating: /content/drive/My Drive/train movielen10k/ml-100k/mku.sh  
  inflating: /content/drive/My Drive/train movielen10k/ml-100k/README  
  inflating: /content/drive/My Drive/train movielen10k/ml-100k/u.data  
  inflating: /content/drive/My Drive/train movielen10k/ml-100k/u.genre  
  inflating: /content/drive/My Drive/train movielen10k/ml-100k/u.info  
  inflating: /content/drive/My Drive/train movielen10k/ml-100k/u.item  
  inflating: /content/drive/My Drive/train movielen10k/ml-100k/u.occupation  
  inflating: /content/drive/My Drive/train movielen10k/ml-100k/u.user  
  inflating: /content/drive/My Drive/train movielen10k/ml-100k/u1.base  
  inflating: /content/drive/My Drive/train movielen10k/ml-100k/u1.test  
  inflating: /content/drive/My Drive/train movielen10k/ml-100k/u2.ba

In [0]:
import pandas as pd
user_cols=['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('/content/drive/My Drive/train movielen10k/ml-100k/u.user',sep="|",names=user_cols,encoding='latin-1',engine='python')
n_users =users.shape[0]
print ('Number of users:', n_users)

Number of users: 943


đầu tiên , nạp các trường của user vào trong file , dùng cb thì chắc cũng chưa cần lắm :v



In [0]:
rate_cols=['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_base = pd.read_csv('ml-100k/ua.base', sep='\t', names=rate_cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', names=rate_cols, encoding='latin-1')
rate_train = ratings_base.as_matrix()
rate_test = ratings_test.as_matrix()
print (rate_train.shape)


(90570, 4)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [0]:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols,
 encoding='latin-1')

In [0]:

X0 = items.as_matrix()
X_train_counts = X0[:, -19:]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [0]:
print (X_train_counts.shape)

(1682, 19)


In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=True, norm ='l2')
tfidf = transformer.fit_transform(X_train_counts.tolist()).toarray()

In [0]:
print (tfidf.shape)

(1682, 19)


In [0]:
import numpy as np
def get_items_rated_by_user(rate_matrix, user_id):
    """
    return (item_ids, scores)
    """
    y = rate_matrix[:,0] 
   
    ids = np.where(y == user_id +1)[0] 
    item_ids = rate_matrix[ids, 1] - 1 
    scores = rate_matrix[ids, 2]
    return (item_ids, scores)

In [0]:
from sklearn.linear_model import Ridge
from sklearn import linear_model

d = tfidf.shape[1] 
W = np.zeros((d, n_users))
b = np.zeros((1, n_users))

for n in range(n_users):    
    ids, scores = get_items_rated_by_user(rate_train, n)
    clf = Ridge(alpha=0.01, fit_intercept  = True)
    Xhat = tfidf[ids, :]
    
    clf.fit(Xhat, scores) 
    W[:, n] = clf.coef_
    b[0, n] = clf.intercept_

In [0]:
Yhat = tfidf.dot(W) + b

In [0]:
from math import sqrt
def evaluate(Yhat, rates, W, b):
    se = 0
    cnt = 0
    for n in range(n_users):
        ids, scores_truth = get_items_rated_by_user(rates, n)
        scores_pred = Yhat[ids, n]
        e = scores_truth - scores_pred 
        se += (e*e).sum(axis = 0)
        cnt += e.size 
    return sqrt(se/cnt)

print ('RMSE for training:', evaluate(Yhat, rate_train, W, b))
print ('RMSE for test    :', evaluate(Yhat, rate_test, W, b))

RMSE for training: 0.908980456282672
RMSE for test    : 1.2703282700393037
